# Assignment 1: Transformations


TEAM-ID:  28 
TEAM-NAME: lilly 
YOUR-ID:   28
YOUR-NAME: Raja 

(Although you work in groups, both the students have to submit to Moodle, hence there's name field above)

# Instructions
- Please check Moodle for "TEAM-ID" and "TEAM-NAME" fields above. Some of your names have been edited because of redundancy/simplicity. Instructions for submitting the assignment through GitHub Classrooms/Moodle has been uploaded on Moodle. Any clarifications will be made there itself.
- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `./misc/installation` for detailed step-by-step instructions about the installation setup.
- Both the team members must submit the zip file.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. Make sure your code is modular since you will be reusing them for future assignments.
- Take a look at the entire assignment. The descriptive questions at the end might have a clue if you are stuck somewhere.
- Answer the descriptive questions in your own words with context & clarity. Do not just copy-paste from some Wikipedia page. You will be evaluated accordingly.
- Please call the visualization functions only when they are asked. They are asked explicitly at the end of every section.
- You could split the Jupyter Notebook cells where `TODO` is written, but please try to avoid splitting/changing the structure of other cells.

In [22]:
import open3d as o3d
import copy
import math
import numpy as np
from scipy.linalg import logm
from scipy.spatial.transform import Rotation as R
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import time

# 1. Getting started with Open3D
## 1.1 Converting RGBD image into Point Cloud


In your robotics journey, it is common to be given just the depth images along with camera parameters in a generic dataset and you'd want to build a 3D data representation out of it, for example, a point cloud. You will understand the math behind these concepts in detail during Vision classes, for now, you can use the in-built functions as a black box.
- Below are the given RGB and D images from SUN RGB-D dataset ([S.Song, CVPR 2015](https://rgbd.cs.princeton.edu/)). 
![low-res-RGBD.resized.png](./misc/low-res-RGBD.png)

- Read these two images `color.jpg` and `depth.png` given in current folder using Open3D. Convert it to a point cloud using the default camera parameters (`o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault`).
Then,
- Create a "world" frame $A$ and combine this (just use $+$ operator) with the above point cloud and save it as `scene.pcd`. Put it aside for now.
- Write a simple function `one_one` to visualize `scene.pcd`.

In [23]:

color_raw = o3d.io.read_image("color.jpg")
depth_raw = o3d.io.read_image("depth.png")
rgbd_image = o3d.geometry.RGBDImage.create_from_sun_format( color_raw, depth_raw)
pcd = o3d.geometry.PointCloud.create_from_rgbd_image( rgbd_image,o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])


geometry::PointCloud with 236957 points.

In [25]:
A = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=0.4,origin=[1,1,1])
pcd1 = A.sample_points_uniformly(number_of_points=5000)
scene = pcd1 + pcd
print(scene)
o3d.io.write_point_cloud("scene.pcd", scene)

geometry::PointCloud with 241957 points.


True

In [ ]:

def one_one(temp):
    o3d.visualization.draw_geometries(
        [ temp])

### Question for 1.1

- In the next code cell, call the function `one_one` here showing `scene.pcd`.

In [ ]:
#uncomment the following and add input parameters if any
one_one(scene)

# 2. Rotations, Euler angles and Gimbal Lock

## 2.1 Rotating an object

The objective here is to roughly simulate an object moving on a ground.

- Generate a cube at some point on the ground and create another frame $B$ at the center of this object. Combine these both as a single point cloud `cube.pcd`. (You can pick a point on the ground by using the `get_picked_points` method of the class `open3d.visualization.VisualizerWithEditing`.)
- Now read both the point clouds `scene.pcd` and `cube.pcd` in a script. Whatever tasks you do below are on the object `cube.pcd` (along with the axes $B$) with `scene.pcd` in the background (static).
- Given a sequence of **ZYX Euler** angles $[30^{\circ}, 90^{\circ}, 45^{\circ}]$, generate the rotation. In our case, our object (with its respective axis) undergoes rotation with the background being fixed (with its respective axis).
- Note: Throughout this assignment, we will be using the standard **ZYX** Euler angle convention.
- Write a function `two_one` to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*

In [ ]:
#vis = o3d.visualization.VisualizerWithEditing()
#vis.create_window()
#vis.add_geometry(scene)
#vis.run()  # user picks points
#vis.destroy_window()
point = [-0.77, -0.94, -3.4]

cub  = o3d.geometry.TriangleMesh.create_box(width=0.4, height=0.4, depth=0.4)
c_t = cub.translate(point)
c_t.paint_uniform_color([0.3, 0.1, 0.1])
p1=cub.get_center()
B = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=0.8,origin=[0,0,0])
B.translate(p1)
C = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=1,origin=[0,0,0])
C.translate(p1)
pcd2 = cub.sample_points_uniformly(number_of_points=400000)
pcd3= B.sample_points_uniformly(number_of_points=400000)




cube= pcd2 + pcd3

o3d.io.write_point_cloud("cube.pcd", cube)
#one_one(cube)


print("Picked point : ",point)

In [ ]:
scene1 = o3d.io.read_point_cloud("scene.pcd")
cube1  = o3d.io.read_point_cloud("cube.pcd")




In [ ]:
def two_one(source,target,frame):    
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(source)
    vis.add_geometry(target)
    vis.add_geometry(frame)    
    R = o3d.geometry.get_rotation_matrix_from_zyx((np.pi/60,0,0))
    for i in range(10):
        source.rotate(R,center=p1)     
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(1/6 )
    
    R_AtoB = o3d.geometry.get_rotation_matrix_from_zyx((np.pi/6,0,0))
    R_AtoC = o3d.geometry.get_rotation_matrix_from_zyx((np.pi/6,(np.pi/20),0))
    R  = np.matmul(R_AtoC,np.transpose(R_AtoB))
    for i in range(10):
        source.rotate(R,center=p1)   
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(1/6 )
    R_AtoB = o3d.geometry.get_rotation_matrix_from_zyx((np.pi/6,np.pi/2,0))
    R_AtoC = o3d.geometry.get_rotation_matrix_from_zyx((np.pi/6,np.pi/2,np.pi/40))
    R  = np.matmul(R_AtoC,np.transpose(R_AtoB))    
    for i in range(10):
        source.rotate(R,center=p1)     
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        time.sleep(1/6 )

### Question for 2.1

- In the next code cell, call the function `two_one` here showing the animation described in section 2.1.

In [ ]:
#uncomment the following and add input parameters if any
two_one(cube1,scene1,C)

## 2.2 Euler angle & Gimbal lock

Code the following yourself from scratch (Refer Craig book - Section: $Z-Y-X$ Euler angles - same conventions/notations followed).

- Case 1: Given the rotation matrix $M_{given}$ below, extract Euler angles $\alpha , \beta ,\gamma$. Convert it back to the rotation matrix $M_{recovered}$ from Euler angles.

    $$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] $$

    After coding it from scratch, check your calculations using `scipy.spatial.transform.Rotation`. (Mandatory)

- Case 2: Given the rotation matrix $N_{given}$, extract Euler angles, and convert back $N_{recovered}$.

    $$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] $$

    Again use `scipy` and check its output. If `scipy` is showing any warnings on any of the above cases, explain it in "**Questions for 2.2**" (last question). Write code in the next cell.
    
- (Optional) Case 3: Do the above two for quaternion using scipy functions, i.e. given the rotation matrix, extract quaternion and convert back.

In [ ]:
##############################################################################
# DON'T EDIT
M_given =  np.array([[0.26200263, -0.19674724, 0.944799],
                     [0.21984631, 0.96542533, 0.14007684],
                     [-0.93969262, 0.17101007, 0.29619813]])

N_given = np.array([[0,-0.173648178,0.984807753],
                    [0, 0.984807753, 0.173648178],
                    [-1, 0, 0]])

In [ ]:
# TODO: Do tasks described in 2.2                                            #
##############################################################################
# Replace "pass" statement with your code
from scipy.spatial.transform import Rotation as Rotation

def RotationMatrix_To_EulerAngles(R) :
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    singular = sy < 1e-6
    if  not singular :
        x = math.atan2( R[2,1] , R[2,2])
        y = math.atan2(-R[2,0] , sy)
        z = math.atan2( R[1,0] , R[0,0])
    else :
        x = math.atan2(-R[1,2] , R[1,1])
        y = math.atan2(-R[2,0] , sy)
        z = 0
    return np.array([z, y, x])



def EulerAngles_To_RotationMatrix(theta) :    
    R_x = np.array([[1,0,0],
                    [0,math.cos(theta[2]),-math.sin(theta[2]) ],
                    [0,math.sin(theta[2]),math.cos(theta[2])  ]
                    ])
    R_y = np.array([[math.cos(theta[1]),0,math.sin(theta[1])  ],
                    [0,1,0],
                    [-math.sin(theta[1]),0,math.cos(theta[1]) ]
                    ])
    R_z = np.array([[math.cos(theta[0]),-math.sin(theta[0]),0 ],
                    [math.sin(theta[0]),math.cos(theta[0]),0],
                    [0,0,1]
                    ])
    R = np.dot(R_z, np.dot( R_y, R_x ))
    return R


r1 = Rotation.from_matrix(M_given)
r2 = Rotation.from_matrix(N_given)

# CASE 1 :
theta1       = RotationMatrix_To_EulerAngles(M_given)
M_recovered  = EulerAngles_To_RotationMatrix(theta1)
Euler1  = theta1*(57.2958)                # converting the radians to Degrees (1 radian = 57.2958 degrees)

# Verification with Scipy
eul1 = r1.as_euler('ZYX', degrees=True)
#print("Angles calculated : ",Euler1)
#print("Angles from scipy : ",eul1)


# CASE 2 :
theta2       = RotationMatrix_To_EulerAngles(N_given)
N_recovered  = EulerAngles_To_RotationMatrix(theta2)
Euler2  = theta2*(57.2958)

# Verification with Scipy
eul2 = r2.as_euler('ZYX', degrees=True)
#print("Angles calculated : ",Euler2)
#print("Angles from scipy : ",eul2)


# CASE 3 :
Quaternion1      = r1.as_quat() 
recovery1        = Rotation.from_quat(Quaternion1)
M_quat_recovered = recovery1.as_matrix()

Quaternion2      = r2.as_quat() 
recovery2        = Rotation.from_quat(Quaternion2)
N_quat_recovered = recovery2.as_matrix()

#print(Quaternion1 )
#print(Quaternion2 )
#print(M_quat_recovered)
#print(N_quat_recovered)
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

## Questions for 2.2
- Have you used `np.arctan` or an any equivalent `atan` function above? Why or why not?   
    * Ans: NO,I have used the function math.atan2, the reason is the functions arctan or any equivalent atan functions has range "-pi/2 to pi/2" which may work for the Given cases but will not work when the angles out of the range so, the function I used "atan2" will have range as "-pi to pi".   

### For Case 1 above,
- What Euler angles  $\alpha , \beta ,\gamma$ did you get? Replace `my_array_case1 ` with your array.

In [ ]:
# Uncomment and replace my_array_case1 with your array.
print("My Euler angles for case 1 are" + str(Euler1))

  - Were you able to recover back your rotation matrix when you converted it from Euler angles? Why/why not? Replace `M_given` and `M_recovered` with your matrices below and explain "why/why not" after this code snippet.

In [ ]:
# Uncomment and Replace M_given and M_recovered with your matrices below.
error = np.linalg.norm(logm(M_given @ M_recovered.T))
print("For case 1, it is " + str(error<0.0001) + " I could recover the original matrix.")

- Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 1? If yes, what is it?

    - Ans:    There is no problem with Euler angle representation for Case 1 as every angle can be uniquely represented

### Repeat the above for Case 2.

In [ ]:
# Uncomment and Replace N_given and N_recovered with your matrices below.
print("My Euler angles for case 2 are" + str(Euler2))
error = np.linalg.norm(logm(N_given @ N_recovered.T))
print("For case 2, it is " + str(error<0.0001) + " I could recover the original matrix.")

* Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 2? If yes, what is it?

    * Ans: Yes , the problem with Case 2 is, it's a Gimbal lock problem.Where we cannot uniquely represent the yaw(about z) and roll(about x) but can only get there sum since the pitch angle is 90 deggress.
    
    
* Explain any more problems with Euler angle representation. Explain what you understand by Gimbal lock (concisely in your own words). You could revisit this question in the section 2.4.
    * Ans: Any orientation sensor or AHRS that uses Euler Angles will always fail to produce reliable estimates when the pitch angle approaches 90 degrees. My understanding of gimbel lock is , we will be finally in a orientation where one axis of rotation coincides with another there by loosing one freedom of rotation , thus any leading to a problem in detecting the orintation of the object. 

- When you used `scipy.spatial.transform.Rotation` for the above 2 cases,
    - Have you used `zyx` above in `r.as_euler('')` argument? Why or why not? Explain the difference between extrinsic and instrinsic rotations with equivalent technical names from Craig book?
        * Ans: I have used 'ZYX' format, beacause 'ZYX' returns the values about the Movable axis(Intrinsic) but the format 'zyx' gives the values w.r.t Fixed axis(Extrinsic).Therefore the difference between Extrinsic and Intrinsic will be Rotaion about, Fixed axis and Movable axis.
    - Has `scipy` shown any warnings on any of the above cases? If yes, explain it.
        * Ans: yes, for the case 2 it shows a warning that gimbal lock is hit, so setting one angle to zero since angles cannot be uniqely represented. The reason is when Gimbal lock is formed we will only be able to know the sum to two rotation angles. There by leading to infinite solutions, so to avoid this and to give a awnser scipy sets one of the angles to zero and the other angle to the known sum.
    - (Optional) For Case 3 above (quaternion) which you did using scipy, did you observe any problem with quaternion? Depending on your observations, which is better? Quaternion or Euler angles? And why?
         * Ans: There are no problems with quaternion representation. Infact it is a compact way of representing a rotation and It will also not have a problem like Gimbal lock with Euler Angles.It will also help in "Smooth Rotation" which has wide range of use. So , the quaternion representation is a prefered and useful way.


## 2.3 Rotation matrix as an Operator
This question will help you in your understanding of [Rotator-transform (Vector-frame) equivalence](https://www.notion.so/saishubodh/Lecture-2-Transformations-11d69d8cef2d4cd195a98fa7d33224e1#f90ece4f5e374743bfed47e46a83ecfe).

![image.png](./misc/xyz-frame.png)
Consider the frame $XYZ$ in the above image. Say you have a vector $x_1=[0,\sqrt{3},0]$. Now you want to rotate it such that you end up at $x_2=[1,1,1]$ through a sequence of Euler angle rotations. Your goal is to find out those $\alpha, \beta \: \& \: \gamma$ ($ZYX$). We will follow this order whenever we refer to it below.

First, properly understand the so-called "Rotator-transform equivalence" to figure out what are the terms of rotation matrix. Then, put the math on paper and you will end up with a set of non-linear equations. Write the set of linear equations in LaTeX here:

$${The\space Equations\space are:}$$
$$\cos(\alpha)*\sin(\beta)*\sin(\gamma)-\sin(\alpha)*\cos(\gamma)-\frac{1}{\sqrt3}=0$$
$$\sin(\alpha)*\sin(\beta)*\sin(\gamma)+\cos(\alpha)*\cos(\gamma)-\frac{1}{\sqrt3}=0$$
$$\cos(\beta)*\sin(\gamma)-\frac{1}{\sqrt3}=0$$

Solve these equations using `fsolve` from `scipy.optimize` as follows: (Come back and answer the following questions after coding it in the next block)
- `case1`: First, solve it with an initialization of (0,0,0). Check if your answer is correct using `np.isclose`.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_1, b_1, c_1$ = `[ -0.39780629 -93.75690183   0.71372211]`* 
- `case2`: Now, forget about the solver for a moment: Can you visualize and think of sequence of rotations one by one to reach the final position (which is different than previous set of rotations)? Now, validate your answer by giving (your answer $\pm 5$) as initialization.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_2, b_2, c_2$ = `[-0.22461136  0.64487776  0.80731668]`*

In [ ]:
##############################################################################
# DON'T EDIT
x_1 = np.array([0,np.sqrt(3),0])
x_2 = np.array([1,1,1])

In [ ]:
##############################################################################
# TODO: Do tasks described in 2.3                                            #
##############################################################################
# Replace "pass" statement with your code
def rmat(x):
    y=[0,0,0]
    y[0]=((math.cos(x[0])*math.sin(x[1])*math.sin(x[2]))-(math.sin(x[0])*math.cos(x[2])))-math.sqrt(1/3)
    y[1]=((math.sin(x[0])*math.sin(x[1])*math.sin(x[2]))+(math.cos(x[0])*math.cos(x[2])))-math.sqrt(1/3)
    y[2]=(math.cos(x[1])*math.sin(x[2]))-math.sqrt(1/3)
    #print("int func")
    #print(y)
    return y
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

In [ ]:
# From Section 2.2, use the function which takes Euler angles and gives Rotation matrix as output.
# Uncomment and replace `r_mat` with the name of the function. (Do NOT edit anything else)
a1=math.radians(-12.86928305)
a2=math.radians(36.94877383)
a3=math.radians(46.25583854)
y=[0,0,0]
x00=fsolve(rmat,[y])
#print(x00)
#y[0]=(math.cos(a1)*math.sin(a2)*math.sin(a3))-(math.sin(a1)*math.cos(a3))*math.sqrt(1)
#y[1]=(math.sin(a1)*math.sin(a2)*math.sin(a3))+(math.cos(a1)*math.cos(a3))*math.sqrt(1)
#y[2]=math.cos(a2)*math.sin(a3)*(math.sqrt(1))
x11=fsolve(rmat,[a1,a2,a3])
#print(x11)
#print(y)
#print(rmat(x00))
x_2_obtained_case1 = EulerAngles_To_RotationMatrix([x00[0],x00[1],x00[2]]) @ x_1.T #TODO: replace r_mat
x_2_obtained_case2 = EulerAngles_To_RotationMatrix([x11[0],x11[1],x11[2]])@ x_1.T #TODO: replace r_mat
#print(x_2_obtained_case1)

test = True #TODO: Set this as True

In [ ]:
# DON'T EDIT
if test == True:
    case1_test = np.isclose(x_2_obtained_case1,  np.array([1.0,1.0,1.0]))
    case2_test = np.isclose(x_2_obtained_case2,  np.array([1.0,1.0,1.0]))
    print("For case 1, it is " + str(bool(case1_test[0] and case1_test[1] and case1_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")
    print("For case 2, it is " + str(bool(case2_test[0] and case2_test[1] and case2_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")

## 2.4 Gimbal Lock visualization (Optional)

A nice visualization video for gimbal lock is [this](https://www.youtube.com/watch?v=zc8b2Jo7mno). You are about to animate a similar visualization demonstrating gimbal lock 😃.

![image.png](./misc/xyz-frame.png)

- Write a function `two_four` for the visualization of gimbal lock. Follow the below steps to get the intuition going. Use Open3D for the following.
    - Say our frame's initial position is as the above image. Now the final goal at the end of rotation is to get the $Y$ axis pointing in the direction of the vector $(x,y,z)$ that you currently see in the above image. This point is fixed in space and is NOT moving as we rotate our axis.
    - For creating that point, you could use a small sphere using `open3d.geometry.create_mesh_sphere`. You already know how to create an axis by now.
    - Following our $ZYX$ convention, first rotate your frame about $Z$ axis by an angle, say $-35^{\circ}$. Then rotate about $Y$ axis by an angle ${\beta}$ and then about $X$ by say $55^{\circ}$.
        - Are there any specific angle(s) $\beta$ using which you will **never** reach our point $(x,y,z)$ ?
            - Clue: We are specifically talking about gimbal lock here and notice the word "never".
        - Under this (these) specific angle(s) of $\beta$ & different combinations of $\alpha$ and $\gamma$, make an animation and clearly show why your $Y$ axis is unable to align in the direction of that vector $(x,y,z)$ using the animation.

            If you are unsure to simulate the animation, you could do it as follows:

            - You could first fix some $\alpha$, say $-35^{\circ}$ & an above value of $\beta$, you can now vary $\gamma$ from $-180^{\circ} \text{ to }180^{\circ}$ to simulate the animation.
            - Now fix another $\alpha$, say $45^{\circ}$ and repeat the above process. So that's 2 specific values of $\alpha$.
            - Show this for all angles of $\beta$ if there are more than 1.
            - Therefore, when the code is run, there should be a minimum of (2 $\times$ (number of values of $\beta$)) animations. 2 for values of $\alpha$, you could show it for even more if you wish to.

In [ ]:
##############################################################################
# TODO: Do tasks described in 2.4                                            #
##############################################################################
# Replace "pass" statement with your code
Global = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=1,origin=[0,0,0])

S  = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)
S.translate([0.25,0.25,0.25])  # locating sphere
S.paint_uniform_color([0.9, 0.1, 0.1])

#Different sets of alpha and gamma
alpha = [-35,30,-50,40,50] 
gama  = [55,-10,70,-20,-30]

def two_four(source,target,frame,j):    
        vis = o3d.visualization.Visualizer()
        vis.create_window()
        vis.add_geometry(source)
        vis.add_geometry(target)
        vis.add_geometry(frame) 
        R = o3d.geometry.get_rotation_matrix_from_zyx((((alpha[j])/1800)*(np.pi),0,0))
        for i in range(10):
            source.rotate(R,center=[0,0,0])     
            vis.update_geometry(source)
            vis.poll_events()
            vis.update_renderer()
            time.sleep(1/6 )    
        R_AtoB = o3d.geometry.get_rotation_matrix_from_zyx((((alpha[j])/180)*(np.pi),0,0))
        R_AtoC = o3d.geometry.get_rotation_matrix_from_zyx((((alpha[j])/180)*(np.pi),(np.pi/20),0))
        R  = np.matmul(R_AtoC,np.transpose(R_AtoB))
        for i in range(10):
            source.rotate(R,center=[0,0,0])   
            vis.update_geometry(source)
            vis.poll_events()
            vis.update_renderer()
            time.sleep(1/6 )
        R_AtoB = o3d.geometry.get_rotation_matrix_from_zyx((((alpha[j])/180)*(np.pi),np.pi/2,0))
        R_AtoC = o3d.geometry.get_rotation_matrix_from_zyx((((alpha[j])/180)*(np.pi),np.pi/2,((gama[j])/1800)*(np.pi)))
        R  = np.matmul(R_AtoC,np.transpose(R_AtoB))    
        for i in range(10):
            source.rotate(R,center=[0,0,0])     
            vis.update_geometry(source)
            vis.poll_events()
            vis.update_renderer()
            time.sleep(1/6 )
        vis.destroy_window()

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

> VOILA! You have just animated the famous Gimbal lock problem. If you are curious, read about the [Apollo 11](https://en.wikipedia.org/wiki/Gimbal_lock#On_Apollo_11) Gimbal lock incident.

### Questions for 2.4 (Optional)

- Mention the value(s) of $\beta$ here: 
    * Ans: *pi/2 , -pi/2.*
- Now that you understand gimbal lock through visualization, explain it now in matrix form: For the above values of $\beta$, what does the rotation matrix look like? Can you explain why gimbal lock occurs from the rotation matrix alone? Clue: Use sin/cos formulae. 
    * Ans: Let $\alpha$,$\beta$,$\gamma$ be the rotations about Z-Y-X then,
    \begin{aligned}R&={\begin{bmatrix}1&0&0\\0&\cos \alpha &-\sin \alpha \\0&\sin \alpha &\cos \alpha \end{bmatrix}}{\begin{bmatrix}\cos \beta &0&\sin \beta \\0&1&0\\-\sin \beta &0&\cos \beta \end{bmatrix}}{\begin{bmatrix}\cos \gamma &-\sin \gamma &0\\\sin \gamma &\cos \gamma &0\\0&0&1\end{bmatrix}}\end{aligned}
    
    For Gimbal lock let, $\beta$ =$\pi$/2. Replacing cos($\pi$/2) = 0 and sin($\pi$/2) = 1, the above expression becomes equal to:
    \begin{aligned}R&={\begin{bmatrix}1&0&0\\0&\cos \alpha &-\sin \alpha \\0&\sin \alpha &\cos \alpha \end{bmatrix}}{\begin{bmatrix}0&0&1\\0&1&0\\-1&0&0\end{bmatrix}}{\begin{bmatrix}\cos \gamma &-\sin \gamma &0\\\sin \gamma &\cos \gamma &0\\0&0&1\end{bmatrix}}\end{aligned}
    On multiplying the matrices finally,
    \begin{aligned}R&={\begin{bmatrix}0&0&1\\\sin \alpha &\cos \alpha &0\\-\cos \alpha &\sin \alpha &0\end{bmatrix}}{\begin{bmatrix}\cos \gamma &-\sin \gamma &0\\\sin \gamma &\cos \gamma &0\\0&0&1\end{bmatrix}}&={\begin{bmatrix}0&0&1\\\sin \alpha \cos \gamma +\cos \alpha \sin \gamma &-\sin \alpha \sin \gamma +\cos \alpha \cos \gamma &0\\-\cos \alpha \cos \gamma +\sin \alpha \sin \gamma &\cos \alpha \sin \gamma +\sin \alpha \cos \gamma &0\end{bmatrix}}\end{aligned}
    And finally using the trigonometry formulas:
    \begin{aligned}R&={\begin{bmatrix}0&0&1\\\sin(\alpha +\gamma )&\cos(\alpha +\gamma )&0\\-\cos(\alpha +\gamma )&\sin(\alpha +\gamma )&0\end{bmatrix}}\end{aligned}
    Changing the values of $\alpha$  and $\gamma$ in the above matrix has the same effects: the rotation angle ($\alpha$ + $\gamma$) changes, but the rotation axis remains in the Z direction: the last column and the first row in the matrix won't change.
    Similarly for $\beta$ =-$\pi$/2 , the rotation angle will be ($\alpha$ - $\gamma$)
.
- Call the function `two_four` for the visualization of gimbal lock written above. 

In [ ]:
#uncomment the following and add input parameters if any
F = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.75,origin=[0,0,0])
two_four(F,S,Global,0)

F1 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.75,origin=[0,0,0])
two_four(F1,S,Global,1)

F2 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.75,origin=[0,0,0])
two_four(F2,S,Global,2)

F3 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.75,origin=[0,0,0])
two_four(F3,S,Global,3)

F4 = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.75,origin=[0,0,0])
two_four(F4,S,Global,4)
